In [ ]:
'''

'''

In [ ]:
'''
    - NLTK -- for tokenezing russian texts of election programs
    
    - (LINK!!) -- for creating matrix of program_x_term
    - sklearn -- for clustering...
'''

# 1. Loading data -- table of election programs

In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
programs_df = pd.read_csv('../library: main/ssmm_elections_candidates_1_11.csv', sep='\t', encoding='utf-8')
programs_df = programs_df[['conv_x_name','program_txt','convocation_no']][programs_df['program_txt'].notna()].reset_index().drop(['index'],axis=1)
programs_df.head()


,conv_x_name,program_txt,convocation_no
0,1_Левин_Андрей,Активный участник и организатор различных меро...,1
1,1_Кириллова_Полина,Мне нравится заниматься общественной деятельно...,1
2,1_Семенов_Павел,"""Здравствуйте. Я так понимаю, что я уже поздно...",1
3,1_Кузнецова_Алёна,"""Меня очень интересует жизнь Университета. я б...",1
4,1_Кунденко_Вадим,""" Добрый день! Пару слов о себе. Путь к мехм...",1


In [3]:
len(programs_df)

481

In [62]:
key_list = list(programs_df['conv_x_name'])[:46]
key_list 

['1_Левин_Андрей',
 '1_Кириллова_Полина',
 '1_Семенов_Павел',
 '1_Кузнецова_Алёна',
 '1_Кунденко_Вадим',
 '1_Сафина_Диана_Ураловна',
 '1_Глухова_Алёна_Олеговна',
 '1_Владыкина_Вероника_Евгеньевна',
 '1_Попов_Леонид_Андреевич',
 '1_Тропин_Александр_Михайлович',
 '1_Сапунов_Кирилл_Вячеславович',
 '1_Артамонова_Мария_Владимировна',
 '1_Удимов_Даниил_Алексеевич',
 '1_Котляров_Никита_Владимирович',
 '1_Чернодед_Антон_Игоревич',
 '1_Штейников_Юрий_Николаевич',
 '1_Савушкин_Никита_Максимович',
 '1_Миценко_Вадим_Валериевич',
 '2_Остроухова_Наталья_Владимировна',
 '2_Габидов_Александр_Сергеевич',
 '2_Дяченко_Мария_Игоревна',
 '2_Ванунц_Арсений_Ашотович',
 '2_Ватутин_Кирилл_Александрович',
 '2_Никифоров_Сергей_Игоревич',
 '2_Власов_Артем_Андреевич',
 '2_Медведева_Яна_Григорьевна',
 '2_Тишин_Павел_Владимирович',
 '2_Багров_Константин_Владимирович',
 '2_Быстрицкая_Василина_Васильевна',
 '2_Сафина_Диана_Ураловна',
 '2_Лавров_Василий_Алексеевич',
 '2_Осин_Руслан_Владимирович',
 '2_Антонов_Сергей_Вал

In [63]:
program_list = list(programs_df['program_txt'])[:46]

# program_list

# 2. Tokenizing of texts

In [7]:
# !pip3 install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 3.4 MB/s eta 0:00:0000:0100:010m
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.4/294.4 kB 1.5 MB/s eta 0:00:00a 0:00:01
  Using cached tqdm-4.64.1-py2.py3-none-any.whl (78 kB)
  Using cached click-8.1.3-py3-none-any.whl (96 kB)


In [10]:
from nltk.tokenize import word_tokenize

In [14]:
import nltk
nltk.download('punkt')
# for russian text

[nltk_data] Downloading package punkt to /Users/user/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [94]:

def remove_noise(text, stop_words = []):
    tokens = word_tokenize(text, language="russian")
    cleaned_tokens = []
    for token in tokens:
        token = re.sub('[^А-Яа-я]+', '', token)
        if len(token) > 1 and token.lower() not in stop_words:
            # Get lowercase
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

# [^A-Za-z0-9]+

In [17]:
# remove_noise("Мне нравится заниматься общественной деятельностью.")

['мне', 'нравится', 'заниматься', 'общественной', 'деятельностью']

In [ ]:
# http://brandonrose.org/clustering#Stopwords,-stemming,-and-tokenizing

In [28]:
nltk.download('stopwords')

stopwords = nltk.corpus.stopwords.words('russian')
stopwords

[nltk_data] Downloading package stopwords to /Users/user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['и',
 'в',
 'во',
 'не',
 'что',
 'он',
 'на',
 'я',
 'с',
 'со',
 'как',
 'а',
 'то',
 'все',
 'она',
 'так',
 'его',
 'но',
 'да',
 'ты',
 'к',
 'у',
 'же',
 'вы',
 'за',
 'бы',
 'по',
 'только',
 'ее',
 'мне',
 'было',
 'вот',
 'от',
 'меня',
 'еще',
 'нет',
 'о',
 'из',
 'ему',
 'теперь',
 'когда',
 'даже',
 'ну',
 'вдруг',
 'ли',
 'если',
 'уже',
 'или',
 'ни',
 'быть',
 'был',
 'него',
 'до',
 'вас',
 'нибудь',
 'опять',
 'уж',
 'вам',
 'ведь',
 'там',
 'потом',
 'себя',
 'ничего',
 'ей',
 'может',
 'они',
 'тут',
 'где',
 'есть',
 'надо',
 'ней',
 'для',
 'мы',
 'тебя',
 'их',
 'чем',
 'была',
 'сам',
 'чтоб',
 'без',
 'будто',
 'чего',
 'раз',
 'тоже',
 'себе',
 'под',
 'будет',
 'ж',
 'тогда',
 'кто',
 'этот',
 'того',
 'потому',
 'этого',
 'какой',
 'совсем',
 'ним',
 'здесь',
 'этом',
 'один',
 'почти',
 'мой',
 'тем',
 'чтобы',
 'нее',
 'сейчас',
 'были',
 'куда',
 'зачем',
 'всех',
 'никогда',
 'можно',
 'при',
 'наконец',
 'два',
 'об',
 'другой',
 'хоть',
 'после',
 'на

In [115]:
# to add stopwords

stopwords += ['это', 'этим', 'свой', 'поэтому', 'многие', 'очень', 'также', 'являюсь'] + [
        'мгу', 'университет', 'университета',
        'студсовета', 'студсовете', 
#         'студентов', 
        'мехмата', 'мехмат',
        'факультета',
        'студентов', 'студенты', 'студентам'
      ]


In [20]:
# Stemming is just the process of breaking a word down into its root.

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("russian")

In [40]:
# sents = nltk.sent_tokenize('Мне нравится заниматься общественной деятельностью, всячески пытаюсь расширять свой кругозор. Еще в школе организовывала различные мероприятия и была сильно этим заинтересована', language="russian")
# print(sents)

# for sent in sents:
#     for word in nltk.word_tokenize(sent, language="russian"):
#         print(word)


In [96]:
#

def tokenize_and_stem(text, stopwords, lang):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text, language=lang) for word in nltk.word_tokenize(sent, language=lang) if word.lower() not in stopwords]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[а-яА-Я]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

def tokenize_only(text, stopwords, lang):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text, language=lang) for word in nltk.word_tokenize(sent, language=lang) if word.lower() not in stopwords]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[а-яА-Я]', token):
            filtered_tokens.append(token)
    return filtered_tokens



In [43]:
# for program in program_list:
#     print(tokenize_and_stem(program, stopwords, "russian"))

In [117]:
#not super pythonic, no, not at all.
#use extend so it's a big flat list of vocab
totalvocab_stemmed = []
totalvocab_tokenized = []

for i in program_list:
    allwords_stemmed = tokenize_and_stem(i, stopwords, "russian") #for each item in 'synopses', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
    allwords_tokenized = tokenize_only(i, stopwords, "russian")
    totalvocab_tokenized.extend(allwords_tokenized)
    
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
print('there are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame')

there are 2873 items in vocab_frame


In [126]:
vocab_frame

,words
активн,активный
участник,участник
организатор,организатор
различн,различных
мероприят,мероприятий
...,...
рабоч,рабочих
групп,групп
образован,образование
бытов,бытовые


# Tf-idf and program similarity

In [ ]:
# key_list, program_list

In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [58]:

#define vectorizer parameters
# tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
#                                  min_df=0.2, stop_words=stopwords,
#                                  use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

# %time tfidf_matrix = tfidf_vectorizer.fit_transform(program_list) 

# print(tfidf_matrix.shape)

In [ ]:
'''
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

max_df - float or int, default=1.0
When building the vocabulary ignore terms that have a document frequency strictly higher 
than the given threshold (corpus-specific stop words). 
If float in range of [0.0, 1.0], the parameter represents a proportion of documents, integer absolute counts. 
This parameter is ignored if vocabulary is not None.

min_df - float or int, default=1
When building the vocabulary ignore terms that have a document frequency strictly lower 
than the given threshold. This value is also called cut-off in the literature. 
If float in range of [0.0, 1.0], the parameter represents a proportion of documents, integer absolute counts. 
This parameter is ignored if vocabulary is not None.

max_features - int, default=None
If not None, build a vocabulary that only consider the top max_features ordered by term frequency across the corpus.
This parameter is ignored if vocabulary is not None.

'''


In [120]:
# alternative way, with won list of stop words
# https://stackoverflow.com/questions/26826002/adding-words-to-stop-words-list-in-tfidfvectorizer-in-sklearn

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(
#     max_df=0.8, max_features=200000, min_df=0.2, 
    ngram_range=(1,1), stop_words=stopwords
)

tfidf_matrix = tfidf_vectorizer.fit_transform(program_list)

# idf_values = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))

# printing the tfidf vectors
# print(tfidf_matrix)

terms = tfidf_vectorizer.get_feature_names_out()

print(tfidf_matrix.shape)


(46, 1758)


# K-Means

In [54]:
from sklearn.cluster import KMeans

In [121]:
num_clusters = 3

km = KMeans(n_clusters=num_clusters)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

CPU times: user 69.3 ms, sys: 56.9 ms, total: 126 ms
Wall time: 41.3 ms


/opt/homebrew/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [122]:
program_clusters = {
    'key': key_list, 
    'program': program_list, 
    'cluster': clusters
}

frame = pd.DataFrame(program_clusters, index = [clusters] , columns = ['key', 'program', 'cluster'])

frame['cluster'].value_counts()

2    24
0    12
1    10
Name: cluster, dtype: int64

In [ ]:
# grouped = frame['rank'].groupby(frame['cluster']) #groupby cluster for aggregation purposes

# grouped.mean() #average rank (1 to 100) per cluster

In [124]:
print("Top terms per cluster:")
print()

#sort cluster centers by proximity to centroid
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

for i in range(num_clusters):
    print("Cluster %d words:" % i, end='')
    
    for ind in order_centroids[i, :6]: #replace 6 with n words per cluster
        print('', terms[ind], end=',')
#         print('', vocab_frame.loc[terms[ind].split(' ')], end=',')
#         print(' %s' % vocab_frame.loc[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore'), end=',')
    print() #add whitespace
    print() #add whitespace
    
    print("Cluster %d keys:" % i, end='')
    for title in frame.loc[i]['key'].values.tolist():
        print(' %s,' % title, end='')
    print() #add whitespace
    print() #add whitespace
    
# print()
# print()

Top terms per cluster:

Cluster 0 words: организацией,

KeyError: "None of [Index(['организацией'], dtype='object')] are in the [index]"